In [1]:
!conda install -y gdown
import gdown

Retrieving notices: ...working... done
Channels:
 - rapidsai
 - nvidia
 - nodefaults
 - conda-forge
 - defaults
 - pytorch
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - gdown


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-24.9.2               |  py310hff52083_0         895 KB  conda-forge
    filelock-3.16.1            |     pyhd8ed1ab_0          17 KB  conda-forge
    gdown-5.2.0                |     pyhd8ed1ab_0          21 KB  conda-forge
    openssl-3.4.0              |       hb9d3cd8_0         2.8 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.7 MB

The following NEW packages will be INSTALLED:

  filelock           conda-forge/noarch::filelock-3.16.1-pyhd8ed1ab_0 
  gdown              conda-forg

In [2]:
train_url = 'https://drive.google.com/uc?id=16434ij3b-G-uIxr1-x8kRq3qRWR_GeAT'
dev_url = 'https://drive.google.com/uc?id=1zOWtEwscZM4x-m39s4Ib9g_OmOQ8inFI'
test_url = 'https://drive.google.com/uc?id=1NqRGgVuiLqbkiV9p-pNvKk3C93EAM-xh'
w2v_url = 'https://drive.google.com/uc?id=1PBiofrHGe4z9tj8X9hjU6WlKk93ZGxHL'

train_out = 'Train_preprocessed_with_-1.csv'
dev_out = 'dev_final.csv'
test_out = 'test_final.csv'
w2v_out = 'W2V_150.txt'

gdown.download(train_url, train_out, quiet=False)
gdown.download(dev_url, dev_out, quiet=False)
gdown.download(test_url, test_out, quiet=False)
gdown.download(w2v_url, w2v_out, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=16434ij3b-G-uIxr1-x8kRq3qRWR_GeAT
To: /kaggle/working/Train_preprocessed_with_-1.csv
100%|██████████| 2.30M/2.30M [00:00<00:00, 161MB/s]
Downloading...
From: https://drive.google.com/uc?id=1zOWtEwscZM4x-m39s4Ib9g_OmOQ8inFI
To: /kaggle/working/dev_final.csv
100%|██████████| 319k/319k [00:00<00:00, 84.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1NqRGgVuiLqbkiV9p-pNvKk3C93EAM-xh
To: /kaggle/working/test_final.csv
100%|██████████| 654k/654k [00:00<00:00, 113MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1PBiofrHGe4z9tj8X9hjU6WlKk93ZGxHL
From (redirected): https://drive.google.com/uc?id=1PBiofrHGe4z9tj8X9hjU6WlKk93ZGxHL&confirm=t&uuid=a3613f0d-f396-41f5-b3d3-c9ee3502397d
To: /kaggle/working/W2V_150.txt
100%|██████████| 119M/119M [00:01<00:00, 72.8MB/s] 


'W2V_150.txt'

In [3]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from torch.nn.functional import softmax
from tqdm import tqdm
from transformers import AutoTokenizer
from sklearn.metrics import precision_score, recall_score, f1_score

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# Define aspect and sentiment mappings
aspect2idx = {
    'CAMERA': 0, 'FEATURES': 1, 'BATTERY': 2, 'PERFORMANCE': 3,
    'DESIGN': 4, 'GENERAL': 5, 'PRICE': 6, 'SCREEN': 7, 'SER&ACC': 8, 'STORAGE': 9
}
sentiment2idx = {
    'Positive': 2, 'Neutral': 1, 'Negative': 0
}
num_aspect = len(aspect2idx)

# Convert label cell to tensor
def convert_label(cell):
    return torch.tensor([float(x) for x in cell.strip('[]').split()])

# Load train data
train = pd.read_csv("Train_preprocessed_with_-1.csv")
sentences_train = list(train['comment'])
labels_train = list(train['label'].apply(convert_label))

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained('bkai-foundation-models/vietnamese-bi-encoder')

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

In [6]:
# Define dataset
class CustomTextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        labels = torch.tensor(self.labels[idx], dtype=torch.float32)
        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        dic = {key: val.squeeze(0) for key, val in encoding.items()}
        dic['labels'] = labels
        return dic

In [7]:
# Create dataset and dataloader
train_dataset = CustomTextDataset(sentences_train, labels_train, tokenizer)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=128)

dev = pd.read_csv('dev_final.csv')
sentences_dev = list(dev['comment'])
labels_dev = list(dev['label'].apply(convert_label))

dev_dataset = CustomTextDataset(sentences_dev, labels_dev, tokenizer)
dev_dataloader = DataLoader(dev_dataset, shuffle=True, batch_size=128)

In [8]:
class AttentionInHtt(nn.Module):
    def __init__(self, in_features, out_features, bias=True, softmax=True):
        super().__init__()
        self.W = nn.Linear(in_features, out_features, bias)
        self.uw = nn.Linear(out_features, 1, bias=False)
        self.softmax = softmax

    def forward(self, h: torch.Tensor, mask: torch.Tensor):
        u = self.W(h)  # (batch_size, seq_len, out_features)
        u = torch.tanh(u)
        similarities = self.uw(u)  # (batch_size, seq_len, 1)
        similarities = similarities.squeeze(dim=-1)  # (batch_size, seq_len)

        # Mask the similarities
        similarities = similarities.masked_fill(~mask.bool(), -float('inf'))

        if self.softmax:
            alpha = torch.softmax(similarities, dim=-1)
            return alpha
        else:
            return similarities
            # return attention score

In [9]:
def element_wise_mul(input1, input2, return_not_sum_result=False):
        output = input1 * input2.unsqueeze(2)  # Ensure correct broadcasting
        result = output.sum(dim=1)
        if return_not_sum_result:
            return result, output
        else:
            return result

In [10]:
class Cae(nn.Module):
    def __init__(self, word_embedder, categories, polarities):
        super().__init__()
        self.word_embedder = word_embedder
        self.category_num = len(categories)
        self.polarity_num = len(polarities)
        self.category_loss = nn.BCEWithLogitsLoss()
        self.sentiment_loss = nn.CrossEntropyLoss(ignore_index=-1)

        embed_dim = word_embedder.embedding_dim
        self.embedding_layer_fc = nn.Linear(embed_dim, embed_dim)
        self.embedding_layer_aspect_attentions = nn.ModuleList([AttentionInHtt(embed_dim, embed_dim) for _ in range(self.category_num)])
        self.lstm_layer_aspect_attentions = nn.ModuleList([AttentionInHtt(embed_dim, embed_dim) for _ in range(self.category_num)])

        self.lstm = nn.LSTM(embed_dim, embed_dim // 2, batch_first=True, bidirectional=True)
        self.dropout_after_embedding = nn.Dropout(0.5)
        self.dropout_after_lstm = nn.Dropout(0.5)

        self.category_fcs = nn.ModuleList([nn.Sequential(nn.Linear(embed_dim * 2, 32), nn.ReLU(), nn.Linear(32, 1)) for _ in range(self.category_num)])
        self.sentiment_fc = nn.Sequential(nn.Linear(embed_dim * 2, 32), nn.ReLU(), nn.Linear(32, self.polarity_num))

    def forward(self, tokens, labels, mask, threshold=0.25):
        word_embeddings = self.word_embedder(tokens)
        word_embeddings = self.dropout_after_embedding(word_embeddings)

        embeddings = word_embeddings
        embedding_layer_category_outputs = []
        embedding_layer_sentiment_outputs = []
        for i in range(self.category_num):
            embedding_layer_aspect_attention = self.embedding_layer_aspect_attentions[i]
            alpha = embedding_layer_aspect_attention(embeddings, mask)

            category_output = element_wise_mul(embeddings, alpha)
            embedding_layer_category_outputs.append(category_output)

            category_output = category_output.unsqueeze(1)
            sentiment_alpha = torch.matmul(category_output, embeddings.transpose(1, 2)).squeeze(1)
            sentiment_alpha = softmax(sentiment_alpha, dim=-1)
            sentiment_output = torch.matmul(sentiment_alpha.unsqueeze(1), word_embeddings).squeeze(1)
            embedding_layer_sentiment_outputs.append(sentiment_output)

        lstm_result, _ = self.lstm(word_embeddings)
        lstm_result = self.dropout_after_lstm(lstm_result)

        lstm_layer_category_outputs = []
        lstm_layer_sentiment_outputs = []
        for i in range(self.category_num):
            lstm_layer_aspect_attention = self.lstm_layer_aspect_attentions[i]
            alpha = lstm_layer_aspect_attention(lstm_result, mask)
            category_output = element_wise_mul(lstm_result, alpha)
            lstm_layer_category_outputs.append(category_output)

            category_output = category_output.unsqueeze(1)
            sentiment_alpha = torch.matmul(category_output, lstm_result.transpose(1, 2)).squeeze(1)
            sentiment_alpha = softmax(sentiment_alpha, dim=-1)
            sentiment_output = torch.matmul(sentiment_alpha.unsqueeze(1), lstm_result).squeeze(1)
            lstm_layer_sentiment_outputs.append(sentiment_output)

        final_category_outputs = []
        final_sentiment_outputs = []
        for i in range(self.category_num):
            fc = self.category_fcs[i]
            category_output = torch.cat([embedding_layer_category_outputs[i], lstm_layer_category_outputs[i]], dim=-1)
            final_category_output = fc(category_output)
            final_category_outputs.append(final_category_output)

            sentiment_output = torch.cat([embedding_layer_sentiment_outputs[i], lstm_layer_sentiment_outputs[i]], dim=-1)
            final_sentiment_output = self.sentiment_fc(sentiment_output)
            final_sentiment_outputs.append(final_sentiment_output)

        loss = 0
        if labels is not None:
            category_labels = labels[:, :self.category_num]
            polarity_labels = labels[:, self.category_num:]

            for i in range(self.category_num):
                category_mask = (category_labels[:, i] != -1)  # Mask out ignored labels
                sentiment_mask = (polarity_labels[:, i] != -1)

                if category_mask.any():  # Only calculate if there are valid labels
                    category_temp_loss = self.category_loss(final_category_outputs[i].squeeze(-1)[category_mask], category_labels[:, i][category_mask])
                    loss += category_temp_loss

                if sentiment_mask.any():  # Only calculate if there are valid labels
                    sentiment_temp_loss = self.sentiment_loss(final_sentiment_outputs[i][sentiment_mask], polarity_labels[:, i][sentiment_mask].long())
                    loss += sentiment_temp_loss

#         output = {
#             'pred_category': [torch.sigmoid(e) for e in final_category_outputs],
#             'pred_sentiment': [torch.softmax(e, dim=-1) for e in final_sentiment_outputs]
#         }
        # formatting output
        final_category_outputs = [torch.sigmoid(e) for e in final_category_outputs]
        final_sentiment_outputs = [torch.softmax(e, dim=-1) for e in final_sentiment_outputs]
        final_sentiment_outputs = [torch.argmax(e, dim=-1) for e in final_sentiment_outputs]
        
        final_categories = []
        final_sentiments = []

        for i in range(len(final_category_outputs)):
            batch_category = []
            batch_sentiment = []
            for j, category_score in enumerate(final_category_outputs[i]):
                # Apply threshold for aspect detection
                if category_score >= threshold:
                    batch_category.append(1)  # Aspect detected
                    batch_sentiment.append(final_sentiment_outputs[i][j].item())
                else:
                    batch_category.append(0)  # Aspect not detected
                    batch_sentiment.append(-1)  # Set sentiment to -1 for undetected aspect
            final_categories.append(batch_category)
            final_sentiments.append(batch_sentiment)
        final_categories = torch.tensor(final_categories)
        final_sentiments = torch.tensor(final_sentiments)
        
        output = {
            'pred_category': torch.transpose(final_categories, 0, 1), # batch_size*10
            'pred_sentiment': torch.transpose(final_sentiments, 0, 1) # batch_size*10
        }

        return output, loss

In [11]:
w2v = 'W2V_150.txt'
embedding_dim = 150
word_to_vec = {}
with open(w2v, 'r', encoding='utf-8') as file:
    for line in file:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        word_to_vec[word] = vector

vocab = tokenizer.get_vocab()
vocab_size = len(vocab)
E = np.zeros((vocab_size, embedding_dim))
for word, idx in vocab.items():
    E[idx] = word_to_vec.get(word, np.random.normal(scale=0.6, size=(embedding_dim,)))

embedding_matrix = torch.tensor(E, dtype=torch.float32)
embedding_layer = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)

In [12]:
categories = aspect2idx.keys()
polarities = sentiment2idx.keys()
model = Cae(embedding_layer, categories, polarities)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

In [13]:
model.to(device)

Cae(
  (word_embedder): Embedding(64001, 150)
  (category_loss): BCEWithLogitsLoss()
  (sentiment_loss): CrossEntropyLoss()
  (embedding_layer_fc): Linear(in_features=150, out_features=150, bias=True)
  (embedding_layer_aspect_attentions): ModuleList(
    (0-9): 10 x AttentionInHtt(
      (W): Linear(in_features=150, out_features=150, bias=True)
      (uw): Linear(in_features=150, out_features=1, bias=False)
    )
  )
  (lstm_layer_aspect_attentions): ModuleList(
    (0-9): 10 x AttentionInHtt(
      (W): Linear(in_features=150, out_features=150, bias=True)
      (uw): Linear(in_features=150, out_features=1, bias=False)
    )
  )
  (lstm): LSTM(150, 75, batch_first=True, bidirectional=True)
  (dropout_after_embedding): Dropout(p=0.5, inplace=False)
  (dropout_after_lstm): Dropout(p=0.5, inplace=False)
  (category_fcs): ModuleList(
    (0-9): 10 x Sequential(
      (0): Linear(in_features=300, out_features=32, bias=True)
      (1): ReLU()
      (2): Linear(in_features=32, out_features=1

In [14]:
# def infer_batch(input_ids, attention_mask, threshold=0.65):
#     # Set the model to evaluation mode
#     model.eval()

#     with torch.no_grad():
#         # Forward pass through the model
#         output, loss = model(tokens=input_ids, labels=None, mask=attention_mask)

#     return {
#         "Detected Aspects": output['pred_category'], # batch_size*10
#         "Aspect Sentiments": output['pred_sentiment'] # batch_size*10
#     }

In [15]:
def infer_single_comment(model, comment):
    model.eval()
    # Tokenize input text
    encoding = tokenizer(comment, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
#     # Prepare mask for polarity prediction
#     batch_size = 1  # Single inference
#     polarity_mask = torch.ones(batch_size, num_aspect).float()

    # No labels provided in inference
    with torch.no_grad():
        output,loss = model(input_ids, labels=None, mask=attention_mask)

    pred_category = output['pred_category']
    pred_sentiment = output['pred_sentiment']
#     # Extract category and sentiment predictions
#     pred_category = [torch.sigmoid(e).item() for e in output['pred_category']]
#     pred_sentiment = [torch.argmax(e).item() for e in output['pred_sentiment']]

    # Map indices to actual labels
    # aspect_labels = list(aspect2idx.keys())
    # sentiment_labels = list(sentiment2idx.keys())
    # results = {
    #     "Aspect": [aspect_labels[i] for i, val in enumerate(pred_category) if val >= 0.5],
    #     "Sentiment": [sentiment_labels[s] for s in pred_sentiment if s > 0]
    # }
    return pred_category,pred_sentiment

In [16]:
# model.eval()
# batch = next(iter(train_dataloader))
# input_ids = batch['input_ids'][0]
# att_mask = batch['attention_mask'][0]
# out = model(input_ids, labels=None, mask=att_mask)
# out['pred_category']
# # metric = calculate_macro_metrics(categories, sentiments, batch['labels'], num_aspect=10)
# # metric

In [17]:
def calculate_macro_metrics(all_pred_categories, all_pred_sentiments, all_true_labels, num_aspect=10):
    """
    Calculate macro-averaged Precision, Recall, and F1-score for Aspect Detection and Sentiment Detection.

    Parameters:
    - all_pred_categories: List of predicted categories (Aspect Detection) for each instance
    - all_pred_sentiments: List of predicted sentiments (Sentiment Detection) for each instance
    - all_true_labels: List of true labels with aspect and sentiment information
    - num_aspect: The number of aspect labels (to split the labels correctly)

    Returns:
    - Dictionary with macro-averaged Precision, Recall, and F1-score for Aspect and Sentiment Detection
    """

    # Separate true labels into aspects and sentiments based on num_aspect
    true_acd = [label[:num_aspect] for label in all_true_labels]  # True Aspect Detection labels
    true_acsa = [label[num_aspect:] for label in all_true_labels]  # True Sentiment Detection labels

    # Flatten lists if needed (this step assumes true_acd and true_acsa are lists of lists)
    true_acd = [item for sublist in true_acd for item in sublist]
    true_acsa = [item for sublist in true_acsa for item in sublist]

    pred_acd = [item for sublist in all_pred_categories for item in sublist]
    pred_acsa = [item for sublist in all_pred_sentiments for item in sublist]

    # Calculate Precision, Recall, and F1-score for Aspect Detection
    acd_precision = precision_score(true_acd, pred_acd, average="macro", zero_division=0)
    acd_recall = recall_score(true_acd, pred_acd, average="macro", zero_division=0)
    acd_f1 = f1_score(true_acd, pred_acd, average="macro", zero_division=0)

    # Calculate Precision, Recall, and F1-score for Sentiment Detection
    acsa_precision = precision_score(true_acsa, pred_acsa, average="macro", zero_division=0)
    acsa_recall = recall_score(true_acsa, pred_acsa, average="macro", zero_division=0)
    acsa_f1 = f1_score(true_acsa, pred_acsa, average="macro", zero_division=0)

    return {
        "Aspect Detection": {
            "Precision": acd_precision * 100,
            "Recall": acd_recall * 100,
            "F1-score": acd_f1 * 100,
        },
        "Sentiment Detection": {
            "Precision": acsa_precision * 100,
            "Recall": acsa_recall * 100,
            "F1-score": acsa_f1 * 100,
        }
    }
    

In [18]:
def eval_dev(model, dev_dataloader):
    model.eval()
    
    with torch.inference_mode():
        loss=0
        pred_cate = []
        pred_sent = []
        true_label = []
        
        for batch in dev_dataloader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)
            att_mask = batch['attention_mask'].to(device)
            output, loss = model(input_ids, labels, att_mask)
            l2_reg = torch.tensor(0., requires_grad=False)  # Initialize L2 regularization term
            for param in model.parameters():
#                 if param not in model.lstm.parameters():
                l2_reg = l2_reg + torch.norm(param, 2)  # Add squared norm of each parameter

            # Add L2 regularization term to loss
            loss = loss + l2_lambda * l2_reg
            
            pred_cate.append(output['pred_category'])
            pred_sent.append(output['pred_sentiment'])
            true_label.append(batch['labels'])
    categories = torch.cat(pred_cate, dim=0)
    sentiments = torch.cat(pred_sent, dim=0)
    labels = torch.cat(true_label, dim=0)
    metric = calculate_macro_metrics(categories, sentiments, labels)
    f1_acd = metric['Aspect Detection']['F1-score']
    f1_sc = metric['Sentiment Detection']['F1-score']
    
    loss = loss / len(dev_dataloader)
    return loss, f1_acd, f1_sc

In [19]:
# l = []
# a = torch.randn((3,10))
# b = torch.randn((3,10))
# l.append(a)
# l.append(b)
# c = torch.cat(l, dim=0)
# c.shape

In [20]:
# Training loop
epochs = 50
l2_lambda = 0.01
for epoch in range(epochs):
    model.train()
    total_loss = 0
    pred_cate = []
    pred_sent = []
    true_label = []
    
    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        att_mask = batch['attention_mask'].to(device)
        output, loss = model(input_ids, labels, att_mask)
#         output, loss = model(batch['input_ids'], batch['labels'], batch['attention_mask'])
        l2_reg = torch.tensor(0., requires_grad=True)  # Initialize L2 regularization term
        for param in model.parameters():
#             if param not in model.lstm.parameters():
            l2_reg = l2_reg + torch.norm(param, 2)  # Add squared norm of each parameter

        # Add L2 regularization term to loss
        loss = loss + l2_lambda * l2_reg
        
        pred_cate.append(output['pred_category'])
        pred_sent.append(output['pred_sentiment'])
        true_label.append(batch['labels'])
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)  # Gradient clipping
        optimizer.step()
        total_loss += loss.item()
        
        
    # metric
    categories = torch.cat(pred_cate, dim=0)
    sentiments = torch.cat(pred_sent, dim=0)
    labels = torch.cat(true_label, dim=0)
#     print(labels.shape)
#     print(categories.shape)
#     print(sentiment.shape)
    metric = calculate_macro_metrics(categories, sentiments, labels)
    f1_acd = metric['Aspect Detection']['F1-score']
    f1_sc = metric['Sentiment Detection']['F1-score']
    dev = eval_dev(model, dev_dataloader)
    
    print(f"Epoch {epoch + 1} - Train_Loss: {total_loss / len(train_dataloader)} - train_acd_f1: {f1_acd} - train_sc_f1: {f1_sc}")
    print(f"dev_loss: {dev[0]} - dev_acd_f1: {dev[1]} - dev_sc_f1: {dev[2]}")
    
    if epoch%10==0 and epoch>0:
        torch.save(model.state_dict(), f"CAE_checkpoint{epoch+1}.pth")

  0%|          | 0/61 [00:00<?, ?it/s]/tmp/ipykernel_31/3054169716.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(self.labels[idx], dtype=torch.float32)
100%|██████████| 61/61 [00:13<00:00,  4.45it/s]
/tmp/ipykernel_31/3054169716.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(self.labels[idx], dtype=torch.float32)


Epoch 1 - Train_Loss: 44.215947885982324 - train_acd_f1: 27.78286958934335 - train_sc_f1: 13.577583442124114
dev_loss: 4.624623775482178 - dev_acd_f1: 52.236938251116236 - dev_sc_f1: 26.021927127646876


  0%|          | 0/61 [00:00<?, ?it/s]/tmp/ipykernel_31/3054169716.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(self.labels[idx], dtype=torch.float32)
100%|██████████| 61/61 [00:11<00:00,  5.22it/s]
/tmp/ipykernel_31/3054169716.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(self.labels[idx], dtype=torch.float32)


Epoch 2 - Train_Loss: 40.951784540395266 - train_acd_f1: 62.50434942159874 - train_sc_f1: 42.45135516543368
dev_loss: 4.440301895141602 - dev_acd_f1: 72.73929684423581 - dev_sc_f1: 53.73066236181517


  0%|          | 0/61 [00:00<?, ?it/s]/tmp/ipykernel_31/3054169716.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(self.labels[idx], dtype=torch.float32)
100%|██████████| 61/61 [00:11<00:00,  5.30it/s]
/tmp/ipykernel_31/3054169716.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(self.labels[idx], dtype=torch.float32)


Epoch 3 - Train_Loss: 38.893184349185134 - train_acd_f1: 72.26480823409528 - train_sc_f1: 52.704943295978524
dev_loss: 4.123592376708984 - dev_acd_f1: 77.49475885416548 - dev_sc_f1: 57.55226716872406


  0%|          | 0/61 [00:00<?, ?it/s]/tmp/ipykernel_31/3054169716.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(self.labels[idx], dtype=torch.float32)
100%|██████████| 61/61 [00:11<00:00,  5.36it/s]
/tmp/ipykernel_31/3054169716.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(self.labels[idx], dtype=torch.float32)


Epoch 4 - Train_Loss: 37.593412055343876 - train_acd_f1: 76.05649047038963 - train_sc_f1: 56.555556616126005
dev_loss: 4.000006198883057 - dev_acd_f1: 80.83204545493314 - dev_sc_f1: 60.63882042210944


  0%|          | 0/61 [00:00<?, ?it/s]/tmp/ipykernel_31/3054169716.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(self.labels[idx], dtype=torch.float32)
100%|██████████| 61/61 [00:11<00:00,  5.36it/s]
/tmp/ipykernel_31/3054169716.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(self.labels[idx], dtype=torch.float32)


Epoch 5 - Train_Loss: 36.63625542062228 - train_acd_f1: 78.29876948044551 - train_sc_f1: 58.48371537253809
dev_loss: 4.053582668304443 - dev_acd_f1: 82.5527155996876 - dev_sc_f1: 61.40835194548882


In [21]:
# torch.save(model.state_dict(), "CAE_ignore_test_lstm.pth")

In [ ]:
# model1 = Cae(embedding_layer, categories, polarities)
# model1.load_state_dict(torch.load('/kaggle/working/CAE_ignore_test_lstm.pth'))
# model1.eval()
# model1.to(device)

In [37]:
# sample_comment = train.iloc[1]['comment']
# x, y = infer_single_comment(model=model, comment=sample_comment)
# print("pred_cate", x)
# print("pred_sent", y) #strict=False

pred_cate tensor([[0, 1, 1, 0, 0, 1, 0, 0, 0, 0]])
pred_sent tensor([[-1,  0,  0, -1, -1,  0, -1, -1, -1, -1]])


In [ ]:
# aspect2idx = {
#     'CAMERA': 0, 'FEATURES': 1, 'BATTERY': 2, 'PERFORMANCE': 3,
#     'DESIGN': 4, 'GENERAL': 5, 'PRICE': 6, 'SCREEN': 7, 'SER&ACC': 8, 'STORAGE': 9
# }
# sentiment2idx = {
#     'Positive': 2, 'Neutral': 1, 'Negative': 0
# }

In [34]:
# train.iloc[1]['label']

'[ 0.  0.  1.  0.  0.  1.  0.  0.  0.  0. -1. -1.  0. -1. -1.  2. -1. -1.\n -1. -1.]'

In [24]:
# model1.eval()

# with torch.no_grad():
#     batch = next(iter(train_dataloader))
#     input_ids = batch['input_ids'].to(device)
#     labels = batch['labels'].to(device)
#     att_mask = batch['attention_mask'].to(device)
#     output, loss = model1(input_ids, labels, att_mask)
#     print(output['pred_category'].shape) 

In [25]:
# x.shape

In [26]:
# đọc lại paper
# check lại data
# xem model build đúng chưa

# viết hàm eval
#   2 task riêng
#   gộp lại
# binary F1 cho ACD
# accuracy cho SC
# F1 cho ACSC

# train model
# print ra metric
# eval treen tap dev

# viết hàm inference


## case studies

## hyperparams
# dropout rate
# embed dim
# clip_norm
# lr, scheduler
# threshold

# exclude lstm
# function for batch